<a href="https://colab.research.google.com/github/Dhnillhaq/Praproses-Ekstraksi-Fitur-Klasifikasi---CM-PCVK-Kel.-5/blob/main/praproses_ekstraksi_fitur_klasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praproses, Ektraksi Fitur, dan ~Klasifikasi~
- ## Kelompok 5, Kelas TI 3-E
    1. Dhanisa Putri Mashilfa
    2. Satrio Wisnu Adi Pratama
    3. Muhamad Syaifullah
    4. Muhammad Dhia Ramadhanil Haq
    5. Muhammad Eril Putra Pratidina


## Persiapan

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2

root = "/content/drive/MyDrive/PBL KEL 5 SEM 5/fruit-clean/Training"
X_train = []
y_train = []

for class_name in os.listdir(root):
    class_folder = os.path.join(root, class_name)
    if not os.path.isdir(class_folder):
        continue

    for file in os.listdir(class_folder):
        img = cv2.imread(os.path.join(class_folder, file))
        img = cv2.resize(img, (128, 128))

        X_train.append(img)
        y_train.append(class_name)


In [ ]:
import os
import cv2

root = "/content/drive/MyDrive/PBL KEL 5 SEM 5/fruit-clean/Test"
X_test = []
y_test = []

for class_name in os.listdir(root):
    class_folder = os.path.join(root, class_name)
    if not os.path.isdir(class_folder):
        continue

    for file in os.listdir(class_folder):
        img = cv2.imread(os.path.join(class_folder, file))
        img = cv2.resize(img, (128, 128))

        X_test.append(img)
        y_test.append(class_name)


## Praproses

### Normalisasi Piksel

In [ ]:
import numpy as np

X_train = np.array(X_train).astype(np.float32) / 255.0

In [ ]:
X_test = np.array(X_test).astype(np.float32) / 255.0

### Konversi ke Grayscale

In [ ]:
X_test_gray = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in X_test])

In [ ]:
X_train_gray = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in X_train])

### Persiapan lainnya

#### Segmentasi

Dataset citra yang digunakan sudah memiliki tampilan gambar yang cukup tepat pada objek.

#### Augmentasi

Dataset citra yang digunakan mengandun sangat banyak data, sehingga tidak diperlukan augmentasi lagi.

## Ekstraksi Fitur

### Local Binary Pattern untuk Tekstur

In [ ]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

def extract_lbp(image, P=8, R=1, method='uniform'):

    # pastikan grayscale
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # hitung LBP
    lbp = local_binary_pattern(image, P, R, method)

    # jumlah pola uniform = P + 2
    bins = int(P + 2)

    # histogram sebagai fitur
    hist, _ = np.histogram(lbp.ravel(),
                           bins=bins,
                           range=(0, bins),
                           density=True)

    return hist  # hasil 1D vector


### Histogram untuk Warna

In [ ]:
def extract_color_hist(img):
    hsv = cv2.cvtColor((img * 255).astype("uint8"), cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0,1,2], None,
                        [8,8,8],
                        [0,180,0,256,0,256])
    return hist.flatten()

### Konkatenasi Fitur

In [ ]:
X_train_features = []

for img in X_train:
    lbp = extract_lbp(img)
    c = extract_color_hist(img)
    features = np.concatenate([lbp, c])
    X_train_features.append(features)

X_train_features = np.array(X_train_features)

/usr/local/lib/python3.12/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


In [ ]:
X_test_features = []

for img in X_test:
    lbp = extract_lbp(img)
    c = extract_color_hist(img)
    features = np.concatenate([lbp, c])
    X_test_features.append(features)

X_test_features = np.array(X_test_features)

/usr/local/lib/python3.12/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
